In [2]:
import pandas as pd
from pylab import *

In [3]:
data_input = '/home/jeffrey_alstott/Dropbox/Patent_centralities/'
data_output = '/home/jeffrey_alstott/technoinnovation/patent_centralities/data/'

In [3]:
patents = pd.read_csv(data_input+'PATENT_INFO.csv', 
                 usecols=['patent_number', 'filing_year', 'mainclass_id'])

/home/jeffrey_alstott/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def floatify(x):
    from numpy import nan
    try:
        return float(x)
    except ValueError:
        return nan

In [5]:
# vc = patents['Class'].value_counts()
# from pylab import array
# vc[array(list(map(type, vc.index.values)))==str].sum()

In [6]:
patents['Class'] = patents['mainclass_id'].apply(floatify)

In [7]:
patents.dropna(inplace=True)

In [8]:
del patents['mainclass_id']

In [7]:
citations = pd.read_csv(data_input+'CITATION_INFO_classes_no_neg_citlag_CLEAN.csv', 
                usecols=['citing_id', 'cited_id', 
                         'filing_year_citing', 'filing_year_cited', 
                        ])

In [8]:
citations.rename(columns={"citing_id": 'Citing_Patent',
                            "cited_id": 'Cited_Patent',
                            "filing_year_citing": 'Year_Citing_Patent',
                            "filing_year_cited": 'Year_Cited_Patent',
                             }, inplace=True)

In [12]:
print(patents.shape)
patents = patents[(patents['patent_number'].isin(citations['Cited_Patent']) | 
                   patents['patent_number'].isin(citations['Citing_Patent'])
                   )]
print(patents.shape)

(5224369, 3)
(5077975, 3)


In [13]:
patents.to_hdf(data_output+'patents.h5', 'df', complevel=9, complib='blosc')

In [5]:
patents.set_index('patent_number', inplace=True)

In [9]:
for patent_type in ['Citing_Patent', 'Cited_Patent']:
    print(patent_type)
    z = empty(citations.shape[0])
    start_ind = 0
    while start_ind<z.shape[0]:
        print(start_ind)
        stop_ind = start_ind+2000000
        z[start_ind:stop_ind] = patents.ix[citations[start_ind:stop_ind][patent_type], 'Class']
        start_ind = stop_ind
    citations['Class_%s'%patent_type] = z

Citing_Patent
0
2000000
4000000
6000000
8000000
10000000
12000000
14000000
16000000
18000000
20000000
22000000
24000000
26000000
28000000
30000000
32000000
34000000
36000000
38000000
40000000
42000000
44000000
46000000
48000000
50000000
52000000
54000000
56000000
Cited_Patent
0
2000000
4000000
6000000
8000000
10000000
12000000
14000000
16000000
18000000
20000000
22000000
24000000
26000000
28000000
30000000
32000000
34000000
36000000
38000000
40000000
42000000
44000000
46000000
48000000
50000000
52000000
54000000
56000000


In [10]:
print(citations.shape)
citations.dropna(inplace=True)
print(citations.shape)

(57770421, 6)
(57656264, 6)


In [12]:
citations.to_hdf(data_output+'citations.h5', 'df', complevel=9, complib='blosc')